# Analyzing Public Sentiments of Chicago Neighborhoods

What is the relationship between neighborhood reputation and context? 
Below I explore this question in the context of Chicago, IL, using social media (Twitter) data and socioeconomic data for each neighborhood.
For this analysis I define "neighborhood" as one of Chicago's 77 community areas, which are officially recognized by the City of Chicago. 
I measure neighborhood reputation based on the sentiment of each tweet that refers to a neighborhood by name (e.g., "Hyde Park"). 

Eventually I'll expand this to mapping the tweets and examining the data for potential correlations with other types of data such as socioeconomics, crime, and types of amenities.


# Step 0: Set up

Importing relevant modules and keys

In [1]:
import time

In [2]:
import pandas as pd

In [ ]:
#%matplotlib inline

In [ ]:
#import datetime

In [3]:
pd.set_option('display.max_colwidth', -1)  # this helps show as much of the tweets in the dataframes as possible

In [4]:
# in terminal: easy_install pip
# in terminal: pip install tweepy
# in terminal: pip install --upgrade pip [per suggestion shown inside terminal]
import tweepy

In [5]:
#import matplotlib.pyplot as plt

In [6]:
# These four codes constitute your authorization to extract data from Twitter
consumer_token = 'qZMmvxcPLfwCixedks1m3jXGg'
consumer_secret = 'UFFyLdOlePodkPYvR6NR64N0SVinVsPezNb1IKg1hNXl06jy67'
access_token = '2382930698-0eCycGIeqv4SUmOvSINQbkhnb2v9hTPDlSpcb8q'
access_token_secret = 'eTmMeL9pj7pcxoupFDqDuNmzWSXUn8UQWrYeYAcKu8xyR'

# OAuth process, using the keys and tokens
auth = tweepy.OAuthHandler(consumer_token, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Creation of the actual interface, using authentication
# api = tweepy.API(auth)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

# Step 1: Get the data

Downloading, cleaning up, and storing Twitter data in a csv file. 

<h3>Step 1.a. Creating three datasets of differing scopes.</h3>

First I experiment with different search criteria. Two focus solely on the Community Area of Englewood, while the third one consists of tweets that mention any of the 77 Community Areas. I use the smallest dataset (#1) to test things out before applying those methods to a larger one.

<h4>Create and save raw dataset (ds) 1:</h4>

Criteria:   
1. list of tweets within last 6-9 days. 
  * This time range limit is set by Twitter, see https://dev.twitter.com/rest/public/search
2. tweet mentions "englewood" AND "chicago"
  * FYI there are two Community Areas inside Chicago with "Englewood" in the name. They're located adjacent to each other. There are also "Englewood" in other states, such as NJ.
3. tweet originates within 15 miles of Chicago, IL
  * Determine this by using geocoding of tweets
  * Identify geocode for Chicago by clicking on latitude/longitude info on Chicago wikipedia page: https://en.wikipedia.org/wiki/Chicago
4. tweet is in English

In [133]:
api.rate_limit_status('search')  # checks the status of my search request limit

{'rate_limit_context': {'access_token': '2382930698-0eCycGIeqv4SUmOvSINQbkhnb2v9hTPDlSpcb8q'},
 'resources': {'search': {'/search/tweets': {'limit': 180,
    'remaining': 180,
    'reset': 1481576765}}}}

In [130]:
# establish datasets or "buckets" into which each relevant data point is going to be appended:
test1_ds1 = []
test2_ds1 = []
test3_ds1 = []
test4_ds1 = []

for tweet in tweepy.Cursor(api.search,
                           q = 'englewood',
                           lang = 'en', 
                           geocode = '41.836944,-87.684722,15mi',
                           show_user=True).items(10):
    test1_ds1.append(tweet.text)
    test2_ds1.append(tweet.created_at)
    test3_ds1.append(tweet.user.followers_count)
    test4_ds1.append(tweet.user.screen_name)

test_df1 = pd.DataFrame({
        "Tweet Text": test1_ds1,
        "Created At": test2_ds1,
        "Current User Followers": test3_ds1,
        "User Screen Name": test4_ds1})

In [132]:
#test_df1

In [138]:
# establish datasets or "buckets" into which each relevant data point is going to be appended:
community_area_name_ds1 = []
user_screen_name_ds1 = []
user_location_ds1 = []  
user_followers_count_ds1 = []
tweet_coordinates_ds1 = []
tweet_date_time_ds1 = [] 
tweet_content_ds1 = [] 
tweet_num_retweet_ds1 = [] 
tweet_num_liked_ds1 = []

for tweet in tweepy.Cursor(api.search,
                           q = 'englewood',
                           lang = 'en', 
#                           geocode = '41.836944,-87.684722,15mi').items(10):
                           geocode = '41.836944,-87.684722,15mi').items():
    community_area_name_ds1.append('Englewood')
    user_screen_name_ds1.append(tweet.user.screen_name)
    user_location_ds1.append(tweet.user.location)
    user_followers_count_ds1.append(tweet.user.followers_count)
    tweet_coordinates_ds1.append(tweet.coordinates)
    tweet_date_time_ds1.append(tweet.created_at)
    tweet_content_ds1.append(tweet.text)
    tweet_num_retweet_ds1.append(tweet.retweet_count)
    tweet_num_liked_ds1.append(tweet.favorite_count)

df1 = pd.DataFrame({
        "Community Area Mentioned": community_area_name_ds1,
        "User Screen Name": user_screen_name_ds1,
        "User Location": user_location_ds1,
        "User Number of Followers":user_followers_count_ds1,
        "Tweet Coordinates": tweet_coordinates_ds1,
        "Tweet Day and Time": tweet_date_time_ds1,
        "Tweet Number of Times Retweeted": tweet_num_retweet_ds1,
        "Tweet Number of Times Liked": tweet_num_liked_ds1,
        "Tweet Content": tweet_content_ds1})

In [139]:
df1.shape

(964, 9)

In [141]:
df1[0:3]

Community Area Mentioned  \
0  Englewood                 
1  Englewood                 
2  Englewood                 

                                                                                                                                   Tweet Content  \
0  Plz help us give a toy 2 a kid in Englewood 4 Xmas\n312-576-2391 we will pick up donations or deliver them to Xperie… https://t.co/OAy0SNqhiU   
1  Pay Family Of Man Killed By Cops $2.34M, Finance Committee Recommends https://t.co/lW19FdWMsw                                                   
2  Pay Family Of Man Killed By Cops $2.34M, Finance Committee Recommends https://t.co/pFOWw1uNwc https://t.co/1ZB0ZJsqzE                           

  Tweet Coordinates  Tweet Day and Time  Tweet Number of Times Liked  \
0  None             2016-12-12 20:36:08  0                             
1  None             2016-12-12 20:33:17  0                             
2  None             2016-12-12 20:33:08  0                             

   Tweet Number of Times Retweeted       User Location  \
0  0                                Englewood, Chicago   
1  0                                Chicago, IL          
2  0                                Chicago, IL          

   User Number of Followers User Screen Name  
0  601                       mystrogrant      
1  215                       CosbyChicago     
2  111                       BurtFuji

In [142]:
df1.to_csv('df1_raw_data.csv')

<h4>Create and save raw dataset 2: "Englewood" AND "Chicago"</h4>

In [15]:
api.rate_limit_status('search')  # checks the status of my search request limit

{'rate_limit_context': {'access_token': '2382930698-0eCycGIeqv4SUmOvSINQbkhnb2v9hTPDlSpcb8q'},
 'resources': {'search': {'/search/tweets': {'limit': 180,
    'remaining': 44,
    'reset': 1481385045}}}}

In [62]:
community_area_name_ds2 = []
coordinates_ds2 = []
date_time_ds2 = []
num_retweet_ds2 = []
num_liked_ds2 = []
tweet_content_ds2 = []

for tweet in tweepy.Cursor(api.search,
                           q = 'englewood' + 'chicago',
                           lang = 'en', 
                           geocode = '41.836944,-87.684722,15mi').items():
    community_area_name2.append('Englewood')
    coordinates_ds2.append(tweet.coordinates)
    date_time_ds2.append(tweet.created_at)
    num_retweet_ds2.append(tweet.retweet_count)
    num_liked_ds2.append(tweet.favorite_count)
    tweet_content_ds2.append(tweet.text)
    
df2 = pd.DataFrame({
        "Community Area Mentioned": community_area_name2,
        "Coordinates": coordinates_ds2,
        "Day and Time": date_time_ds2,
        "Number of Times Retweeted": num_retweet_ds2,
        "Number of Times Liked": num_liked_ds2,
        "Tweet Content": tweet_content_ds2})

In [63]:
df2.shape

(2, 5)

In [66]:
df2

Coordinates        Day and Time  Number of Times Liked  \
0  None       2016-12-09 15:16:19  0                       
1  None       2016-12-01 15:06:34  1                       

   Number of Times Retweeted  \
0  0                           
1  0                           

                                                                                                                                Tweet Content  
0  @TherealTaraji we love you Taraji # youth \n#Rise Foundation \n#EnglewoodChicago                                                            
1  I'm just a product of what made me⭐️😷                                                            #EnglewoodChicago https://t.co/nNdnJfys1E

In [20]:
# df2.to_csv('df2_raw_data.csv')  

# When I assure that the tweet uses both "Englewood" and "Chicago," then I get only 1-2 tweets for the last 6-9 days.
# For the rest of this notebook, therefore, I will leave df2 behind and continue to work with only df1 
# (because it's small enough for experimenting) and df3 (which covers every community area).

<h4>Create and save raw dataset 3: any of the 77 community area names AND within 15mi of Chicago</h4>

In [ ]:
# Dataset criteria: 
# list of tweets within last 6-9 days
# tweet mentions the name of one of 77 community areas
# tweet originates within 15 miles of Chicago, IL
# tweet is in English

In [68]:
# for list of community area names, importing csv file from previous homework assignment
SES = pd.read_csv('https://raw.githubusercontent.com/yarikan/final_project_in_progress/master/Census_Data_-_Selected_socioeconomic_indicators_in_Chicago__2008___2012.csv?token=AQAvwOm8ph3Ee0gA0SmW9m7zM5JecEtJks5YVFjYwA%3D%3D')
SES = SES.dropna()
SES
# SES.columns
# SES['COMMUNITY AREA NAME']

Community Area Number     COMMUNITY AREA NAME  PERCENT OF HOUSING CROWDED  \
0   1.0                    Rogers Park             7.7                          
1   2.0                    West Ridge              7.8                          
2   3.0                    Uptown                  3.8                          
3   4.0                    Lincoln Square          3.4                          
4   5.0                    North Center            0.3                          
5   6.0                    Lake View               1.1                          
6   7.0                    Lincoln Park            0.8                          
7   8.0                    Near North Side         1.9                          
8   9.0                    Edison Park             1.1                          
9   10.0                   Norwood Park            2.0                          
10  11.0                   Jefferson Park          2.7                          
11  12.0                   Forest Glen             1.1                          
12  13.0                   North Park              3.9                          
13  14.0                   Albany Park             11.3                         
14  15.0                   Portage Park            4.1                          
15  16.0                   Irving Park             6.3                          
16  17.0                   Dunning                 5.2                          
17  18.0                   Montclaire              8.1                          
18  19.0                   Belmont Cragin          10.8                         
19  20.0                   Hermosa                 6.9                          
20  21.0                   Avondale                6.0                          
21  22.0                   Logan Square            3.2                          
22  23.0                   Humboldt park           14.8                         
23  24.0                   West Town               2.3                          
24  25.0                   Austin                  6.3                          
25  26.0                   West Garfield Park      9.4                          
26  27.0                   East Garfield Park      8.2                          
27  28.0                   Near West Side          3.8                          
28  29.0                   North Lawndale          7.4                          
29  30.0                   South Lawndale          15.2                         
..   ...                              ...           ...                         
47  48.0                   Calumet Heights         2.1                          
48  49.0                   Roseland                2.5                          
49  50.0                   Pullman                 1.5                          
50  51.0                   South Deering           4.0                          
51  52.0                   East Side               6.8                          
52  53.0                   West Pullman            3.3                          
53  54.0                   Riverdale               5.8                          
54  55.0                   Hegewisch               3.3                          
55  56.0                   Garfield Ridge          2.6                          
56  57.0                   Archer Heights          8.5                          
57  58.0                   Brighton Park           14.4                         
58  59.0                   McKinley Park           7.2                          
59  60.0                   Bridgeport              4.5                          
60  61.0                   New City                11.9                         
61  62.0                   West Elsdon             11.1                         
62  63.0                   Gage Park               15.8                         
63  64.0                   Clearing                2.7                          
64  65.0                   West Law

In [ ]:
community_area_name_ds3 = []
coordinates_ds3 = []
date_time_ds3 = []
num_retweet_ds3 = []
num_liked_ds3 = []
tweet_content_ds3 = []

for name in SES['COMMUNITY AREA NAME']:
    time.sleep(20)
    for tweet in tweepy.Cursor(api.search,
                               q = name,
                               geocode = "41.836944,-87.684722,15mi",
                               lang = "en").items():
        community_area_name_ds3.append(name.lower())
        coordinates_ds3.append(tweet.coordinates)
        date_time_ds3.append(tweet.created_at)
        num_retweet_ds3.append(tweet.retweet_count)
        num_liked_ds3.append(tweet.favorite_count)
        tweet_content_ds3.append(tweet.text)

df3 = pd.DataFrame({
        "Community Area Mentioned": community_area_name_ds3,
        "Coordinates": coordinates_ds3,
        "Day and Time": date_time_ds3,
        "Number of Times Retweeted": num_retweet_ds3,
        "Number of Times Liked": num_liked_ds3,
        "Tweet Content": tweet_content_ds3})

In [139]:
df3.shape

(1691, 14)

In [140]:
df3

Unnamed: 0  Unnamed: 0.1  Unnamed: 0.1 Community Area Mentioned  \
0              0             0             0              rogers park   
1              1             1             1              rogers park   
2              2             2             2              rogers park   
3              3             3             3              rogers park   
4              4             4             4              rogers park   
5              5             5             5              rogers park   
6              6             6             6              rogers park   
7              7             7             7              rogers park   
8              8             8             8              rogers park   
9              9             9             9              rogers park   
10            10            10            10              rogers park   
11            11            11            11              rogers park   
12            12            12            12              rogers park   
13            13            13            13              rogers park   
14            14            14            14              rogers park   
15            15            15            15              rogers park   
16            16            16            16              rogers park   
17            17            17            17              rogers park   
18            18            18            18              rogers park   
19            19            19            19              rogers park   
20            20            20            20              rogers park   
21            21            21            21              rogers park   
22            22            22            22              rogers park   
23            23            23            23              rogers park   
24            24            24            24               west ridge   
25            25            25            25               west ridge   
26            26            26            26                   uptown   
27            27            27            27                   uptown   
28            28            28            28                   uptown   
29            29            29            29                   uptown   
...          ...           ...           ...                      ...   
1661        1661          1661          1661                   o'hare   
1662        1662          1662          1662                   o'hare   
1663        1663          1663          1663                   o'hare   
1664        1664          1664          1664                edgewater   
1665        1665          1665          1665                edgewater   
1666        1666          1666          1666                edgewater   
1667        1667          1667          1667                edgewater   
1668        1668          1668          1668                edgewater   
1669        1669          1669          1669                edgewater   
1670        1670          1670          1670                edgewater   
1671        1671          1671          1671                edgewater   
1672        1672          1672          1672                edgewater   
1673        1673          1673          1673                edgewater   
1674        1674          1674          1674                edgewater   
1675        1675          1675          1675                edgewater   
1676        1676          1676          1676                edgewater   
1677        1677          1677          1677                edgewater   
1678        1678          1678          1678                edgewater   
1679        1679          1679          1679                edgewater   
1680        1680          1680          1680                edgewater   
1681        1681          1681          1681                edgewater   
1682        1682          1682          1682                edgewater   
1683        1683          1683          1683                edge

In [29]:
df3.to_csv('df3_raw_data.csv')  # resulted in 1,691 tweets on 3/17/16

# Step 2: Do sentiment analysis of tweets


In [7]:
df1 = pd.read_csv('https://raw.githubusercontent.com/yarikan/final_project_in_progress/master/df1_raw_data.csv?token=AQAvwAvWcbuCGIeknomQgS0IPKZmQ_1Rks5YVX7OwA%3D%3D')

In [8]:
df1.columns

Index(['Unnamed: 0', 'Community Area Mentioned', 'Tweet Content',
       'Tweet Coordinates', 'Tweet Day and Time',
       'Tweet Number of Times Liked', 'Tweet Number of Times Retweeted',
       'User Location', 'User Number of Followers', 'User Screen Name'],
      dtype='object')

In [9]:
import nltk

In [10]:
# in command line, pip install twython 
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [11]:
analyzer = SentimentIntensityAnalyzer()
df1["Sentiment"] = df1['Tweet Content'].apply(lambda tweet: analyzer.polarity_scores(tweet))

In [13]:
df1[0:1]

Unnamed: 0 Community Area Mentioned  \
0  0           Englewood                 

                                                                                                                                   Tweet Content  \
0  Plz help us give a toy 2 a kid in Englewood 4 Xmas\n312-576-2391 we will pick up donations or deliver them to Xperie… https://t.co/OAy0SNqhiU   

  Tweet Coordinates   Tweet Day and Time  Tweet Number of Times Liked  \
0  NaN               2016-12-12 20:36:08  0                             

   Tweet Number of Times Retweeted       User Location  \
0  0                                Englewood, Chicago   

   User Number of Followers User Screen Name  \
0  601                       mystrogrant       

                                                      Sentiment  
0  {'pos': 0.174, 'neg': 0.0, 'compound': 0.4588, 'neu': 0.826}

In [14]:
df1.Sentiment[0]

# The following examples demonstrate the value of the compound score:
# VADER is smart, handsome, and funny. {'neg': 0.0, 'neu': 0.254, 'pos': 0.746, 'compound': 0.8316} 
# VADER is smart, handsome, and funny! {'neg': 0.0, 'neu': 0.248, 'pos': 0.752, 'compound': 0.8439} 
# VADER is very smart, handsome, and funny. {'neg': 0.0, 'neu': 0.299, 'pos': 0.701, 'compound': 0.8545} 
# VADER is VERY SMART, handsome, and FUNNY. {'neg': 0.0, 'neu': 0.246, 'pos': 0.754, 'compound': 0.9227} 
# VADER is VERY SMART, handsome, and FUNNY!!! {'neg': 0.0, 'neu': 0.233, 'pos': 0.767, 'compound': 0.9342} 
# VADER is VERY SMART, really handsome, and INCREDIBLY FUNNY!!! {'neg': 0.0, 'neu': 0.294, 'pos': 0.706, 'compound': 0.9469} 

{'compound': 0.4588, 'neg': 0.0, 'neu': 0.826, 'pos': 0.174}

In [15]:
df1['Sentiment'][0]

{'compound': 0.4588, 'neg': 0.0, 'neu': 0.826, 'pos': 0.174}

In [16]:
print(type(df1['Sentiment'][0]))

<class 'dict'>


In [17]:
# convert to string
df1['Sentiment'] = df1['Sentiment'].apply(str)

In [18]:
print(type(df1['Sentiment'][0]))

<class 'str'>


<h4>Extracting sentiment compound scores as additional column</h4>

In [19]:
df1_temp = df1['Sentiment'].str.split(',').apply(pd.Series)
df1_temp[0:5]

0              1                     2               3
0  {'pos': 0.174   'neg': 0.0     'compound': 0.4588    'neu': 0.826}
1  {'pos': 0.113   'neg': 0.351   'compound': -0.6124   'neu': 0.536}
2  {'pos': 0.107   'neg': 0.331   'compound': -0.6124   'neu': 0.562}
3  {'pos': 0.0     'neg': 0.507   'compound': -0.802    'neu': 0.493}
4  {'pos': 0.0     'neg': 0.474   'compound': -0.802    'neu': 0.526}

In [20]:
df1_temp[0][0:5]

0    {'pos': 0.174
1    {'pos': 0.113
2    {'pos': 0.107
3    {'pos': 0.0  
4    {'pos': 0.0  
Name: 0, dtype: object

In [21]:
left_split = df1_temp[0].str.split(': ').apply(pd.Series)
left_split[1][0:5]

0    0.174
1    0.113
2    0.107
3    0.0  
4    0.0  
Name: 1, dtype: object

In [22]:
compound = left_split[1].apply(float)
df1['Compound Sentiment Score'] = compound

In [23]:
df1.columns

Index(['Unnamed: 0', 'Community Area Mentioned', 'Tweet Content',
       'Tweet Coordinates', 'Tweet Day and Time',
       'Tweet Number of Times Liked', 'Tweet Number of Times Retweeted',
       'User Location', 'User Number of Followers', 'User Screen Name',
       'Sentiment', 'Compound Sentiment Score'],
      dtype='object')

In [24]:
df1[0:5]

Unnamed: 0 Community Area Mentioned  \
0  0           Englewood                 
1  1           Englewood                 
2  2           Englewood                 
3  3           Englewood                 
4  4           Englewood                 

                                                                                                                                   Tweet Content  \
0  Plz help us give a toy 2 a kid in Englewood 4 Xmas\n312-576-2391 we will pick up donations or deliver them to Xperie… https://t.co/OAy0SNqhiU   
1  Pay Family Of Man Killed By Cops $2.34M, Finance Committee Recommends https://t.co/lW19FdWMsw                                                   
2  Pay Family Of Man Killed By Cops $2.34M, Finance Committee Recommends https://t.co/pFOWw1uNwc https://t.co/1ZB0ZJsqzE                           
3  Man killed in Englewood shooting, crash identified https://t.co/u4zp50KKYz #chicago                                                             
4  Man killed in Englewood shooting, crash identified... #news #Chicago https://t.co/gOxJDsJOiB                                                    

  Tweet Coordinates   Tweet Day and Time  Tweet Number of Times Liked  \
0  NaN               2016-12-12 20:36:08  0                             
1  NaN               2016-12-12 20:33:17  0                             
2  NaN               2016-12-12 20:33:08  0                             
3  NaN               2016-12-12 20:30:07  0                             
4  NaN               2016-12-12 20:28:02  0                             

   Tweet Number of Times Retweeted       User Location  \
0  0                                Englewood, Chicago   
1  0                                Chicago, IL          
2  0                                Chicago, IL          
3  0                                Chicago, Illinois    
4  0                                Chicago, IL          

   User Number of Followers User Screen Name  \
0  601                       mystrogrant       
1  215                       CosbyChicago      
2  111                       BurtFuji          
3  19150                     chicagonewsnow    
4  127                       GeosNewsChicago   

                                                         Sentiment  \
0  {'pos': 0.174, 'neg': 0.0, 'compound': 0.4588, 'neu': 0.826}      
1  {'pos': 0.113, 'neg': 0.351, 'compound': -0.6124, 'neu': 0.536}   
2  {'pos': 0.107, 'neg': 0.331, 'compound': -0.6124, 'neu': 0.562}   
3  {'pos': 0.0, 'neg': 0.507, 'compound': -0.802, 'neu': 0.493}      
4  {'pos': 0.0, 'neg': 0.474, 'compound': -0.802, 'neu': 0.526}      

   Compound Sentiment Score  
0  0.174                     
1  0.113                     
2  0.107                     
3  0.000                     
4  0.000

In [160]:
df1.to_csv('df1_wCompound.csv')

In [137]:
#df3_temp = df3['Sentiment'].str.split(',').apply(pd.Series)
#left_split = df3_temp[3].str.split(': ').apply(pd.Series)
#compound = left_split[1].apply(float)
#df3['Compound'] = compound
#df3.to_csv('df3_wCompound.csv')

# Step 3: Descriptive Statistics


In [25]:
df1 = pd.read_csv('https://raw.githubusercontent.com/yarikan/final_project_in_progress/master/df1_wCompound.csv?token=AQAvwAyoKxP3pVfAj5BdS7tCHl1C4B6Vks5YWCR_wA%3D%3D')
#df3 = pd.read_csv('https://raw.githubusercontent.com/yarikan/assignments/master/Final_Project/df3_wCompound.csv?token=AQAvwO0ktyQ0b5QB5tjc9kAJoazk7EnVks5W9J9RwA%3D%3D')

In [26]:
df1.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Community Area Mentioned',
       'Tweet Content', 'Tweet Coordinates', 'Tweet Day and Time',
       'Tweet Number of Times Liked', 'Tweet Number of Times Retweeted',
       'User Location', 'User Number of Followers', 'User Screen Name',
       'Sentiment', 'Compound Sentiment Score'],
      dtype='object')

In [27]:
del df1['Unnamed: 0']
del df1['Unnamed: 0.1']

In [37]:
df1['Count 1'] = 1

In [38]:
df1.shape

(964, 16)

In [30]:
# computing an index for neighborhood sentiment:
# for the purpose of this exercise, I assume that followers, users who "like" a tweet, and users who retweet it 
# each play an additive role in tweet influence. I compute their impact separately below even though in reality 
# they may overlap in terms of users.

# compound sentiment score * user number of followers = exposure1 
df1['User to Follower Diffusion'] = df1['Compound Sentiment Score']*df1['User Number of Followers']

# compound sentiment score * tweet number of likes = exposure2
df1['User to Liker Diffusion'] = df1['Compound Sentiment Score']*df1['Tweet Number of Times Liked']

# compound sentiment score * tweet number of retweeted = exposure3
df1['User to Retweeter Diffusion'] = df1['Compound Sentiment Score']*df1['Tweet Number of Times Retweeted']

# exposure1 + exposure2 + exposure3 = total level of diffussion of ONE tweet's sentiment, tweet may or may not be a retweet of another 
df1['Tweet Sentiment Total Diffusion'] = df1['User to Follower Diffusion'] + df1['User to Liker Diffusion'] + df1['User to Retweeter Diffusion']

In [39]:
df1[0:5]

Community Area Mentioned  \
0  Englewood                 
1  Englewood                 
2  Englewood                 
3  Englewood                 
4  Englewood                 

                                                                                                                                   Tweet Content  \
0  Plz help us give a toy 2 a kid in Englewood 4 Xmas\n312-576-2391 we will pick up donations or deliver them to Xperie… https://t.co/OAy0SNqhiU   
1  Pay Family Of Man Killed By Cops $2.34M, Finance Committee Recommends https://t.co/lW19FdWMsw                                                   
2  Pay Family Of Man Killed By Cops $2.34M, Finance Committee Recommends https://t.co/pFOWw1uNwc https://t.co/1ZB0ZJsqzE                           
3  Man killed in Englewood shooting, crash identified https://t.co/u4zp50KKYz #chicago                                                             
4  Man killed in Englewood shooting, crash identified... #news #Chicago https://t.co/gOxJDsJOiB                                                    

  Tweet Coordinates   Tweet Day and Time  Tweet Number of Times Liked  \
0  NaN               2016-12-12 20:36:08  0                             
1  NaN               2016-12-12 20:33:17  0                             
2  NaN               2016-12-12 20:33:08  0                             
3  NaN               2016-12-12 20:30:07  0                             
4  NaN               2016-12-12 20:28:02  0                             

   Tweet Number of Times Retweeted       User Location  \
0  0                                Englewood, Chicago   
1  0                                Chicago, IL          
2  0                                Chicago, IL          
3  0                                Chicago, Illinois    
4  0                                Chicago, IL          

   User Number of Followers User Screen Name  \
0  601                       mystrogrant       
1  215                       CosbyChicago      
2  111                       BurtFuji          
3  19150                     chicagonewsnow    
4  127                       GeosNewsChicago   

                                                         Sentiment  \
0  {'compound': 0.4588, 'neu': 0.826, 'neg': 0.0, 'pos': 0.174}      
1  {'compound': -0.6124, 'neu': 0.536, 'neg': 0.351, 'pos': 0.113}   
2  {'compound': -0.6124, 'neu': 0.562, 'neg': 0.331, 'pos': 0.107}   
3  {'compound': -0.802, 'neu': 0.493, 'neg': 0.507, 'pos': 0.0}      
4  {'compound': -0.802, 'neu': 0.526, 'neg': 0.474, 'pos': 0.0}      

   Compound Sentiment Score  User to Follower Diffusion  \
0  0.4588                    275.7388                     
1 -0.6124                   -131.6660                     
2 -0.6124                   -67.9764                      
3 -0.8020                   -15358.3000                   
4 -0.8020                   -101.8540                     

   User to Liker Diffusion  User to Retweeter Diffusion  \
0  0.0                      0.0                           
1 -0.0                     -0.0                           
2 -0.0                     -0.0                           
3 -0.0                     -0.0                           
4 -0.0                     -0.0                           

   Tweet Sentiment Total Diffusion  Count 1  
0  275.7388                         1        
1 -131.6660                         1        
2 -67.9764                          1        
3 -15358.3000                       1        
4 -101.8540                         1

In [52]:
# compute the number of tweets (total, positive, negative) for neighborhood:

# total number of tweets:
tweet_neighb = df1.groupby('Community Area Mentioned')
tweet_total_neighb = tweet_neighb['Community Area Mentioned'].agg('count')
tweet_total_neighb

Community Area Mentioned
Englewood    964
Name: Community Area Mentioned, dtype: int64

In [53]:
tweet_neighb = df1.groupby('Community Area Mentioned')
# total number of positive tweets:
total_tweet_neighb_pos = tweet_neighb.apply(lambda x: x[x['Compound Sentiment Score'] > 0]['Count 1'].sum())
# total number of negative tweets:
total_tweet_neighb_neg = tweet_neighb.apply(lambda x: x[x['Compound Sentiment Score'] < 0]['Count 1'].sum())
# total number of neutral tweets:
total_tweet_neighb_neu = tweet_neighb.apply(lambda x: x[x['Compound Sentiment Score'] == 0]['Count 1'].sum())
# total number of tweets:
total_tweet_neighb = total_tweet_neighb_pos + total_tweet_neighb_neg + total_tweet_neighb_neu

print(total_tweet_neighb_pos)
print(total_tweet_neighb_neg)
print(total_tweet_neighb_neu)
print(total_tweet_neighb)

Community Area Mentioned
Englewood    243
dtype: int64
Community Area Mentioned
Englewood    379
dtype: int64
Community Area Mentioned
Englewood    342
dtype: int64
Community Area Mentioned
Englewood    964
dtype: int64


In [54]:
tweet_neighb = df1.groupby('Community Area Mentioned')
# total positive tweet influence:
total_tweet_neighb_pos_infl = tweet_neighb.apply(lambda x: x[x['Compound Sentiment Score'] > 0]['Tweet Sentiment Total Diffusion'].sum())
# total negative tweet influence:
total_tweet_neighb_neg_infl = tweet_neighb.apply(lambda x: x[x['Compound Sentiment Score'] < 0]['Tweet Sentiment Total Diffusion'].sum())
# total neutral tweet influence:
total_tweet_neighb_neu_infl = tweet_neighb.apply(lambda x: x[x['Compound Sentiment Score'] == 0]['Tweet Sentiment Total Diffusion'].sum())

print(total_tweet_neighb_pos_infl)
print(total_tweet_neighb_neg_infl)
print(total_tweet_neighb_neu_infl)

Community Area Mentioned
Englewood    909049.2071
dtype: float64
Community Area Mentioned
Englewood   -4.238438e+06
dtype: float64
Community Area Mentioned
Englewood    0.0
dtype: float64


In [56]:
((5*(total_tweet_neighb_neg_infl))+total_tweet_neighb_pos_infl)/(total_tweet_neighb_pos + total_tweet_neighb_neg)
# but this is missing entirely all neutral tweets. is that the right way to think about it? 

Community Area Mentioned
Englewood   -32609.551602
dtype: float64

In [ ]:
#for score in df1['Compound Sentiment Score']:
#    if score < 0:
#        print(score)

<h4>Computing weighted sum of sentiments per neighborhood, giving more weight to negative tweets </h4>
Based on HBR article (Folkman, Jack Zenger and Joseph. 2013. “The Ideal Praise-to-Criticism Ratio.” Harvard Business Review. March 15. https://hbr.org/2013/03/the-ideal-praise-to-criticism.), ideal praise-to-criticism ratio = 5-6 positive comments for every negative one: assume 5 positive for every 1 negative. 

In [32]:
sum_pos_sentiment_diffusion = 0
sum_neg_sentiment_diffusion = 0

for score in df1['Compound Sentiment Score']:
    if score > 0:
        sum_pos_sentiment_diffusion += score
    if score < 0:
        sum_neg_sentiment_diffusion += score

print(sum_pos_sentiment_diffusion)
print(sum_neg_sentiment_diffusion)

121.6472
-184.4313


In [241]:
# [weight * sum of positive sentiments for neighborhood] + [weight * sum of negative sentiments for neighborhood] = weighted total sentiment for neighborhood
neighborhood_sentiment_diffusion_index = (0.2*sum_pos_sentiment_diffusion) + (1*sum_neg_sentiment_diffusion)
neighborhood_sentiment_diffusion_index 

-160.10185999999976

In [242]:
df1

Community Area Mentioned  \
0    Englewood                 
1    Englewood                 
2    Englewood                 
3    Englewood                 
4    Englewood                 
5    Englewood                 
6    Englewood                 
7    Englewood                 
8    Englewood                 
9    Englewood                 
10   Englewood                 
11   Englewood                 
12   Englewood                 
13   Englewood                 
14   Englewood                 
15   Englewood                 
16   Englewood                 
17   Englewood                 
18   Englewood                 
19   Englewood                 
20   Englewood                 
21   Englewood                 
22   Englewood                 
23   Englewood                 
24   Englewood                 
25   Englewood                 
26   Englewood                 
27   Englewood                 
28   Englewood                 
29   Englewood                 
..         ...                 
934  Englewood                 
935  Englewood                 
936  Englewood                 
937  Englewood                 
938  Englewood                 
939  Englewood                 
940  Englewood                 
941  Englewood                 
942  Englewood                 
943  Englewood                 
944  Englewood                 
945  Englewood                 
946  Englewood                 
947  Englewood                 
948  Englewood                 
949  Englewood                 
950  Englewood                 
951  Englewood                 
952  Englewood                 
953  Englewood                 
954  Englewood                 
955  Englewood                 
956  Englewood                 
957  Englewood                 
958  Englewood                 
959  Englewood                 
960  Englewood                 
961  Englewood                 
962  Englewood                 
963  Englewood                 

                                                                                                                                             Tweet Content  \
0    Plz help us give a toy 2 a kid in Englewood 4 Xmas\n312-576-2391 we will pick up donations or deliver them to Xperie… https://t.co/OAy0SNqhiU           
1    Pay Family Of Man Killed By Cops $2.34M, Finance Committee Recommends https://t.co/lW19FdWMsw                                                           
2    Pay Family Of Man Killed By Cops $2.34M, Finance Committee Recommends https://t.co/pFOWw1uNwc https://t.co/1ZB0ZJsqzE                                   
3    Man killed in Englewood shooting, crash identified https://t.co/u4zp50KKYz #chicago                                                                     
4    Man killed in Englewood shooting, crash identified... #news #Chicago https://t.co/gOxJDsJOiB                                                            
5    UPDATE: Man killed in Englewood shooting, crash identified as 31-year-old Nathaniel Lewis-Edwards. https://t.co/axQB8Rnzga                              
6    Pay Family Of Man Killed By Cops $2.34M, Finance Committee Recommends https://t.co/mSg4tkAgD7 https://t.co/C92tUmlk1E                                   
7    Volunteers from all over the city came out to help @MBMHMC clear snow for seniors in Englewood this weekend https://t.co/E52A6UjWWb                     
8    Fund brings ‘Quality of Life’ to Englewood https://t.co/z7WViMcvlq via @CCChronicle                                                                     
9    @EricDickerson for Rams head coach! 👍🏽🙌🏽👊🏽                                                                                                              
10   Retweeted MetroFamily Services (@MetroFamChicago):\n\nPeople from all over Chicago come to help @MBMHMC shovel... https://t.co/L7MfjHQ5Yv               
11   RT @MetroFamChicago: People from all over Chicago come to help @MBMHMC shovel sidewalks for #Englewood 's elderly http